In [5]:
import pandas as pd
from itertools import combinations

import time
import warnings
from tqdm.notebook import tqdm
import os

from nltk.tokenize import RegexpTokenizer
from collections import Counter
import re

In [58]:
# !pip install pandas
# !pip install nltk
# !pip install ipywidgets==7.7.0

In [6]:
# Settings that can be changed
BASEFOLDER = '../../../../data/enwiki_extracted/'
# corpus_name = 'city_matrix.csv'
# new_corpus = False

cities_df = pd.read_csv(f'../../input/List_of_cities_300k.csv', delimiter=';')

window_size = 0 # Set to Paragraph

# Change these (three) variables if you want to process more cities!

In [16]:
OUTPUT_PATH = '../../../../enwiki_city_pairs/biggest_cities_30/'
updated_nr_of_cities = 30 # by population numbers
current_nr_of_cities = 20

In [25]:
updated_list_of_cities = list(cities_df['Mua_en'][0:updated_nr_of_cities].str.lower())
updated_city_pairs = list(combinations(updated_list_of_cities, 2))
sorted(updated_list_of_cities)

['athens',
 'barcelona',
 'berlin',
 'birmingham',
 'brussels',
 'bucharest',
 'budapest',
 'cologne',
 'copenhagen',
 'frankfurt',
 'glasgow',
 'hamburg',
 'katowice',
 'lisbon',
 'london',
 'lyon',
 'madrid',
 'manchester',
 'milan',
 'munich',
 'naples',
 'paris',
 'prague',
 'rome',
 'stockholm',
 'stuttgart',
 'turin',
 'valencia',
 'vienna',
 'warsaw']

In [18]:
current_list_of_cities = list(cities_df['Mua_en'][0:current_nr_of_cities].str.lower())
current_city_pairs = list(combinations(current_list_of_cities, 2))

In [19]:
# len(list(combinations([x for x in range(200)], 2))) Shows number of city pairs
print(f'''Adding '{updated_nr_of_cities-current_nr_of_cities}' cities --> From {current_nr_of_cities} to {updated_nr_of_cities} cities.
Adding '{len(updated_city_pairs)-len(current_city_pairs)}' city pairs --> From {len(current_city_pairs)} to {len(updated_city_pairs)} city pairs.
''')

Adding '10' cities --> From 20 to 30 cities.
Adding '245' city pairs --> From 190 to 435 city pairs.



In [20]:
CITY_PAIRS = [item for item in updated_city_pairs if item not in current_city_pairs]
len(CITY_PAIRS)

245

In [15]:
from IPython.display import clear_output
from ipywidgets import Output
from IPython.display import display
out = Output()
display(out)

import sys
tokenizer = RegexpTokenizer('\w+')

def process_wikidump(basefolder=BASEFOLDER,
                city_pairs = CITY_PAIRS):

    warnings.simplefilter('ignore')
    folders = [f.path for f in os.scandir(basefolder) if f.is_dir()]
    i = 0
    for folder in tqdm(folders, total=len(folders), leave=True, desc='Folders'):
        i+=1
        subfolders = [ f.path for f in os.scandir(folder) if f.is_dir()]
        for subfolder in tqdm(subfolders, total=len(subfolders), leave=False, desc='Sub Folders'):
            if '_V4' in subfolder:
                continue

            for file in os.listdir(subfolder): # tqdm(os.listdir(subfolder), total=len(os.listdir(subfolder)), leave=True, desc='Text Files'):                            
                with open(os.path.join(subfolder, file), "r", encoding="utf-8") as f:
                    process_file(file, f.read(), city_pairs) 
                with out:
                    print(subfolder)
                    clear_output(wait=True)

            os.replace(subfolder, subfolder + '_V4')

        if i == 10:
            break



def process_file(filename, file_content, city_pairs):
    articles = file_content.split('</doc>')
    articles = [x.strip() for x in articles if len(x.strip())]
    
    for article in articles:
        for paragraph in article.split('\n'):
            tokenized_paragraph = tokenizer.tokenize(paragraph.lower())
        
            word_count = Counter(tokenized_paragraph)
            words = set(word_count.keys())
            
            for city_pair in city_pairs:
                detected_cities = words.intersection(set(city_pair))

                if len(detected_cities) == 2:
                    reg_str = "title=\"" + "(.*?)" + "\""
                    title = re.findall(reg_str, article)

                    reg_str = "id=\"" + "(.*?)" + "\""
                    article_id = re.findall(reg_str, article)
                    
                    with out:
                        print('\r',filename," || ", "title: ", title[0], " || ", "citypair: ", city_pair, end='                                            ')

                    content = f'title="{title[0]}", id={article_id[0]} \n{paragraph} \n\n'

                    textfile = f'{OUTPUT_FOLDER}{city_pair[0]}_{city_pair[1]}.txt'

                    with open(textfile, 'a+', encoding='utf-16') as f:
                        f.writelines(content)

Output()

In [106]:
%%time

protection = input(f"Are you sure you want to collect window text of {updated_nr_of_cities} cities? (currently: '{current_nr_of_cities}' cities).\nType 'Yes' to continue.")

if protection == 'Yes':
    print()
    t = 5
    while t:
        print(f"\rcontinuing processing in {t}...", end="")
        t -= 1
        time.sleep(1)
    clear_output()
    # process_wikidump(basefolder=BASEFOLDER, city_pairs=CITY_PAIRS)
else:
    print("\nCanceling Operation...\n")

Are you sure you want to collect window text of 20 cities? (currently: '10' cities).
Type 'Yes' to continue.n

Canceling Operation...

CPU times: total: 31.2 ms
Wall time: 964 ms


<hr>
<hr>
<hr>
<hr>
<hr>
<hr>
<hr>
<hr>
<!-- <br>
<br>
<br>
<br>
<br>
<br> -->
<div style="height: 500px;"></div>